In [10]:
import torch
import torchvision
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

In [11]:
train_data=torchvision.datasets.CIFAR10(root='./datasets',train=True,download=True,transform=transforms.ToTensor())
test_data=torchvision.datasets.CIFAR10(root='./datasets',train=False,download=True,transform=transforms.ToTensor())

In [12]:
len(train_data),len(test_data)

(50000, 10000)

In [13]:
class myModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=nn.Sequential(
        nn.Conv2d(3,32,5,padding=2),
        nn.MaxPool2d(2),
        nn.Conv2d(32,32,5,padding=2),
        nn.MaxPool2d(2),
        nn.Conv2d(32,64,5,padding=2),
        nn.MaxPool2d(2),
        nn.Flatten(),
        nn.Linear(1024,64),
        nn.Linear(64,10))
    def forward(self,x):
        return self.model(x)

In [14]:
train_data_loader=DataLoader(train_data,batch_size=64,shuffle=True)
test_data_loader=DataLoader(test_data,batch_size=64,shuffle=True)

In [15]:
model=myModel()
loss_fn=nn.CrossEntropyLoss()
learning_rate=1e-2
epoch=10
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [16]:
total_train_times=0
total_test_times=0

In [17]:
writer=SummaryWriter('log8')

In [ ]:
for i in range(1,epoch+1):
    print(f'-------------epoch:{i} begining---------------')
    for data in train_data_loader:
        imgs,labels=data
        outputs=model(imgs)
        loss=loss_fn(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_times+=1
        if total_train_times%100==0:
            print('total_train_times:',total_train_times,'total_loss:',loss)
    total_loss=0
    with torch.no_grad():
        for data in test_data_loader:
            imgs,labels=data
            outputs=model(imgs)
            loss=loss_fn(outputs,labels)
            total_loss+=loss
    total_test_times+=1
    print('total_loss:',total_loss)
    writer.add_scalar('test_loss',total_loss,total_train_times)

-------------epoch:1 begining---------------
total_train_times: 100 total_loss: tensor(2.2936, grad_fn=<NllLossBackward0>)
total_train_times: 200 total_loss: tensor(2.2707, grad_fn=<NllLossBackward0>)
total_train_times: 300 total_loss: tensor(2.2235, grad_fn=<NllLossBackward0>)
